In [1]:
import numpy as np
from tabulate import tabulate

In [2]:
import sqlite3
db = sqlite3.connect('test1')
cursor = db.cursor()

In [3]:
import random

In [4]:
def createRelation(relationName, schema, data):
    fieldNames = schema[0]
    fieldTypes = schema[1]
    primaryKeyIndex = schema[2]
    foreignKeys = schema[3]
    cursor.execute(
    f'''
    DROP TABLE IF EXISTS {relationName};
    ''')
    argString = ''
    for i, name in enumerate(fieldNames):
        argString += f'{name} {fieldTypes[i]}'
        if primaryKeyIndex != None:
            if i == primaryKeyIndex:
                argString += ' PRIMARY KEY'
        if i < len(fieldNames) - 1:
            argString += ', '
    for i, v in enumerate(foreignKeys):
        argString += f', FOREIGN KEY({fieldNames[v[0]]}) REFERENCES {v[1]}({fieldNames[v[0]]})'
    cursor.execute(
    f'''
    CREATE TABLE {relationName} ({argString});
    ''')
    for i, r in enumerate(data):
        argString = ''
        for i, e in enumerate(r):
            if isinstance(e, str):
                argString += f"'{e}'"
            else:
                argString += str(e)
            if i < len(fieldNames) - 1:
                argString += ', '
        cursor.execute(
        f'''
        INSERT INTO {relationName} values ({argString});
        ''')
    cursor.execute(f'SELECT * FROM {relationName}')
    output = cursor.fetchall()
    print(relationName)
    showRelation(output, fieldNames)
    return output

def showRelation(data, headers):
    print(tabulate(data, headers, tablefmt="fancy_grid"))
    return

def runCode(statementString):
    cursor.execute(statementString)
    data = cursor.fetchall()
    showRelation(data, [description[0] for description in cursor.description])
    return data

def createSchema(fieldNames, fieldTypes, primaryKeyIndex = None, foreignKeys = []):
    return [fieldNames, fieldTypes, primaryKeyIndex, foreignKeys]

def newCode():
    print("runCode('''\n\n''')")

In [5]:
# DnD expectation
Die1Schema = createSchema(['die1'], ['INTEGER'], 0)
Die1Data = [[1],[2],[3],[4],[5],[6]]
createRelation('Die1', Die1Schema, Die1Data)

Die2Schema = createSchema(['die2'], ['INTEGER'], 0)
Die2Data = [[1],[2],[3],[4],[5],[6]]
createRelation('Die2', Die2Schema, Die2Data)

Die3Schema = createSchema(['die3'], ['INTEGER'], 0)
Die3Data = [[1],[2],[3],[4],[5],[6]]
createRelation('Die3', Die3Schema, Die3Data)

Die4Schema = createSchema(['die4'], ['INTEGER'], 0)
Die4Data = [[1],[2],[3],[4],[5],[6]]
createRelation('Die4', Die4Schema, Die4Data)

Die1
╒════════╕
│   die1 │
╞════════╡
│      1 │
├────────┤
│      2 │
├────────┤
│      3 │
├────────┤
│      4 │
├────────┤
│      5 │
├────────┤
│      6 │
╘════════╛
Die2
╒════════╕
│   die2 │
╞════════╡
│      1 │
├────────┤
│      2 │
├────────┤
│      3 │
├────────┤
│      4 │
├────────┤
│      5 │
├────────┤
│      6 │
╘════════╛
Die3
╒════════╕
│   die3 │
╞════════╡
│      1 │
├────────┤
│      2 │
├────────┤
│      3 │
├────────┤
│      4 │
├────────┤
│      5 │
├────────┤
│      6 │
╘════════╛
Die4
╒════════╕
│   die4 │
╞════════╡
│      1 │
├────────┤
│      2 │
├────────┤
│      3 │
├────────┤
│      4 │
├────────┤
│      5 │
├────────┤
│      6 │
╘════════╛


[(1,), (2,), (3,), (4,), (5,), (6,)]

In [20]:
# Expected value when you roll 4 D6 dice, take sum of best 3
dice = 4
a1 = []
a2 = []
for i in range(1, dice + 1):
    a1.append(f'die{i}')
    a2.append(f'Die{i}')
s1 = ', '.join(a1)
s2 = ' CROSS JOIN '.join(a2)
s3 = ' + '.join(a1)

runCode('''
WITH z AS
(SELECT ''' + s1 + '''
FROM ''' + s2 + '''),
y AS
(SELECT ''' + s1 + ''', ''' + s3 + ''' AS sum, MIN(''' + s1 + ''') AS min
FROM z),
x AS
(SELECT ''' + s1 + ''', sum, min, sum - min AS best3
FROM y),
w AS
(SELECT best3, COUNT(best3) AS occ
FROM x
GROUP BY best3),
v AS
(SELECT best3, occ, CAST(occ AS FLOAT(24)) / CAST(1296 AS FLOAT(24)) AS chance
FROM w),
u AS
(SELECT AVG(best3) AS answer
FROM x)
SELECT *
FROM u
''')

╒══════════╕
│   answer │
╞══════════╡
│  12.2446 │
╘══════════╛


[(12.244598765432098,)]

In [45]:
# Expected value when you do the procedure above 6 times and choose the best one
dice = 4
a1 = []
a2 = []
for i in range(1, dice + 1):
    a1.append(f'die{i}')
    a2.append(f'Die{i}')
s1 = ', '.join(a1)
s2 = ' CROSS JOIN '.join(a2)
s3 = ' + '.join(a1)

times = 6
s4 = ''
a5 = []
a6 = []
a7 = []
for i in range(1, times + 1):
    s4 += f'try{i} AS\n(SELECT best3 AS b{i}, chance AS chance{i} FROM v),\n'
    a5.append(f'b{i}')
    a6.append(f'chance{i}')
    a7.append(f'try{i}')
s5 = ', '.join(a5)
s6 = ' * '.join(a6)
s7 = ' CROSS JOIN '.join(a7)

runCode('''
WITH z AS
(SELECT ''' + s1 + '''
FROM ''' + s2 + '''),
y AS
(SELECT ''' + s1 + ''', ''' + s3 + ''' AS sum, MIN(''' + s1 + ''') AS min
FROM z),
x AS
(SELECT ''' + s1 + ''', sum, min, sum - min AS best3
FROM y),
w AS
(SELECT best3, COUNT(best3) AS occ
FROM x
GROUP BY best3),
v AS
(SELECT best3, occ, CAST(occ AS FLOAT(24)) / CAST(1296 AS FLOAT(24)) AS chance
FROM w),''' + s4 + '''
u AS
(SELECT ''' + s5 + ''', ''' + s6 + ''' AS weight
FROM ''' + s7 + '''),
t AS
(SELECT MAX(''' + s5 + ''') AS best, weight
FROM u),
s AS
(SELECT best, best * weight AS summand
FROM t),
r AS
(SELECT SUM(summand) AS answer
FROM s)
SELECT *
FROM r
''')

╒══════════╕
│   answer │
╞══════════╡
│  15.6619 │
╘══════════╛


[(15.661886272452945,)]

In [62]:
# Simulations

# Roll 4 D6 dice, take sum of best 3
n = 10 ** 5
dice = [0] * 4
s = []
for i in range(n):
    for j in range(len(dice)):
        dice[j] = random.randint(1,6)
    s.append(sum(dice) - min(dice))
print(sum(s) / n)

# Do the procedure above 6 times and choose the best one
n = 10 ** 4
b = []
for k in range(n):
    t = 6
    dice = [0] * 4
    s = []
    for i in range(t):
        for j in range(len(dice)):
            dice[j] = random.randint(1,6)
        s.append(sum(dice) - min(dice))
    b.append(max(s))
print(sum(b) / n)

12.25374
15.6436
